<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/oddiya_video_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 2. Gemini API 라이브러리 설치
!pip install -q google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import subprocess
import shutil
import json
import google.generativeai as genai
from google.colab import userdata

# --- 설정 ---
# 1. Colab 보안 비밀에서 API 키를 안전하게 가져옵니다.
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

# 2. 영상의 주제를 간단하게 입력하세요.
VIDEO_THEME = "제주도의 아름다운 풍경과 맛있는 음식들"

# 3. 사진이 저장된 구글 드라이브 폴더 경로
image_folder = '/content/drive/MyDrive/images'

# 4. 최종적으로 저장될 영상 파일 이름
output_video = 'output_shorts_with_effects.mp4'

# 5. 영상 속성
IMAGE_DURATION = 3
TRANSITION_DURATION = 1
TARGET_FPS = 30 # 목표 프레임 레이트

# 6. AI에게 알려줄 FFmpeg 전환 효과 목록
AVAILABLE_TRANSITIONS = [
    'fade', 'fadeblack', 'fadewhite', 'distance', 'wipeleft', 'wiperight', 'wipeup', 'wipedown',
    'slideleft', 'slideright', 'slideup', 'slidedown', 'circleopen', 'circleclose', 'dissolve',
    'pixelize', 'radial', 'diagtl', 'diagtr', 'diagbl', 'diagbr'
]

# --- 로직 ---
print("🚀 영상 생성을 시작합니다...")

# Gemini API 설정
model = None
if GEMINI_API_KEY:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel('gemini-2.0-flash')
        print("✅ Gemini API 설정 완료.")
    except Exception as e:
        print(f"❗️ API 설정 중 오류 발생: {e}")
else:
    print("❗️ Colab 보안 비밀(Secrets)에서 'GOOGLE_API_KEY'를 찾을 수 없습니다.")

# 이미지 파일 목록 가져오기
image_files = sorted(glob.glob(os.path.join(image_folder, '*.jpeg')))
num_images = len(image_files)

if num_images < 2:
    print(f"❗️'{image_folder}' 폴더에 최소 2개 이상의 .jpeg 파일이 필요합니다.")
else:
    # AI 콘텐츠 생성
    ai_content = {}
    if model:
        try:
            print("🤖 AI가 제목과 전환 효과를 생성 중입니다...")
            num_transitions = num_images - 1
            prompt = f"""'{VIDEO_THEME}' 주제의 영상에 사용할 제목과 전환 효과를 추천해줘.
            - 전환 효과는 다음 목록에서만 {num_transitions}개를 순서대로 골라줘: {', '.join(AVAILABLE_TRANSITIONS)}
            - 답변은 반드시 아래 JSON 형식으로만 해줘:
            {{
              "title": "Your Title Here",
              "transitions": ["effect1", "effect2", ...]
            }}"""
            response = model.generate_content(prompt)
            json_str = response.text.strip().split('```json')[1].split('```')[0]
            ai_content = json.loads(json_str)
            while len(ai_content.get('transitions', [])) < num_transitions:
                ai_content.get('transitions', []).append('fade')
        except Exception as e:
            print(f"❗️ AI 콘텐츠 생성 중 오류 발생: {e}")
            ai_content = {}

    if not ai_content:
        video_title = "My Awesome Slideshow"
        transitions = ['fade'] * (num_images - 1)
        print("⚠️ AI 콘텐츠 생성에 실패하여 기본 제목과 'fade' 효과를 사용합니다.")
    else:
        video_title = ai_content.get('title', "AI Title Failed")
        transitions = ai_content.get('transitions', ['fade'] * (num_images - 1))

    # --- 순차적 영상 생성 로직 ---
    temp_dir = "temp_processing"
    if os.path.exists(temp_dir): shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    try:
        # 1. 첫 이미지로 기반 영상 생성
        print("🎬 1단계: 기반 영상 생성 중...")
        base_video_path = os.path.join(temp_dir, "step_0.mp4")
        command = [
            'ffmpeg', '-y', '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_files[0],
            '-vf', f"scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS}",
            '-c:v', 'libx264', '-r', str(TARGET_FPS), base_video_path
        ]
        subprocess.run(command, check=True, capture_output=True, text=True)

        # 2. 나머지 이미지를 순차적으로 합성
        for i in range(1, num_images):
            print(f"🎬 {i+1}단계: {i+1}번째 이미지 합성 중 (효과: {transitions[i-1]})...")
            input_video = base_video_path
            output_video_temp = os.path.join(temp_dir, f"step_{i}.mp4")

            video_duration = (i * IMAGE_DURATION) - ((i - 1) * TRANSITION_DURATION)

            # ⭐️ 수정된 부분: 새로 들어오는 이미지([1:v])에도 fps 필터를 추가하여 프레임 레이트를 통일시킵니다.
            filter_complex_str = (
                f"[0:v]settb=AVTB[v0];"
                f"[1:v]scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2,setsar=1,format=yuv420p,fps={TARGET_FPS},settb=AVTB[v1];"
                f"[v0][v1]xfade=transition={transitions[i-1]}:duration={TRANSITION_DURATION}:offset={video_duration - TRANSITION_DURATION}"
            )

            command = [
                'ffmpeg', '-y', '-i', input_video, '-loop', '1', '-t', str(IMAGE_DURATION), '-i', image_files[i],
                '-filter_complex', filter_complex_str,
                '-c:v', 'libx264', '-r', str(TARGET_FPS), output_video_temp
            ]
            subprocess.run(command, check=True, capture_output=True, text=True)
            base_video_path = output_video_temp

        # 3. 최종 결과물을 원하는 파일명으로 복사
        shutil.copy(base_video_path, output_video)

        print("\n🎉 영상 생성 완료! 🎉")
        print("="*40)
        print(f"🎬 AI 추천 제목: {video_title}")
        print(f"✨ 적용된 효과: {', '.join(transitions)}")
        print(f"💾 저장된 파일: {output_video}")
        print("="*40)
        print("왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.")

    except subprocess.CalledProcessError as e:
        print("\n❗️ FFmpeg 실행 중 오류가 발생했습니다.")
        print("--- FFmpeg Command ---")
        print(' '.join(f"'{arg}'" if ' ' in arg else arg for arg in command))
        print("\n--- FFmpeg Error ---")
        print(e.stderr)
    finally:
        # 4. 임시 파일 정리
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

🚀 영상 생성을 시작합니다...
✅ Gemini API 설정 완료.
🤖 AI가 제목과 전환 효과를 생성 중입니다...
🎬 1단계: 기반 영상 생성 중...
🎬 2단계: 2번째 이미지 합성 중 (효과: fade)...
🎬 3단계: 3번째 이미지 합성 중 (효과: wipeleft)...
🎬 4단계: 4번째 이미지 합성 중 (효과: slideright)...
🎬 5단계: 5번째 이미지 합성 중 (효과: dissolve)...

🎉 영상 생성 완료! 🎉
🎬 AI 추천 제목: 제주, 맛과 멋에 취하다: 눈과 입이 즐거운 제주 여행
✨ 적용된 효과: fade, wipeleft, slideright, dissolve
💾 저장된 파일: output_shorts_with_effects.mp4
왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.


In [ ]:

# # 1. Gemini API 설정
# try:
#     genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
#     model = genai.GenerativeModel('gemini-1.5-flash')
#     print("✅ Gemini API 설정 완료.")
# except Exception as e:
#     print(f"❗️ API 키 설정에 실패했습니다. 키를 확인해주세요: {e}")
#     model = None

✅ Gemini API 설정 완료.


In [ ]:
# # 2. AI로 영상 제목 생성
# video_title = "제목 생성 실패"
# if model:
#     try:
#         print("🤖 AI가 영상 제목을 생성 중입니다...")
#         prompt = f"'{VIDEO_THEME}'라는 주제의 쇼츠 영상에 어울리는, 짧고 임팩트 있는 제목 하나만 추천해줘."
#         response = model.generate_content(prompt)
#         video_title = response.text.strip().replace("*", "").replace("\"", "") # 특수문자 제거
#     except Exception as e:
#         print(f"❗️ AI 제목 생성 중 오류 발생: {e}")

🤖 AI가 영상 제목을 생성 중입니다...


In [ ]:
# # 3. 이전 작업 파일이 남아있다면 깨끗하게 정리
# if os.path.exists(temp_folder):
#     shutil.rmtree(temp_folder)
# os.makedirs(temp_folder)

In [ ]:
# # 4. 지정된 폴더에서 모든 .jpeg 파일 목록을 가져와 정렬
# image_files = sorted(glob.glob(os.path.join(image_folder, '*.jpeg')))

In [ ]:

# if not image_files:
#     print(f"❗️'{image_folder}' 폴더에 .jpeg 파일이 없습니다. 경로를 확인해주세요.")
# else:
#     # 5. FFmpeg가 인식하기 쉽도록 파일을 숫자 순서대로 복사
#     for i, file_path in enumerate(image_files):
#         shutil.copy(file_path, os.path.join(temp_folder, f"img{i:03d}.jpeg"))

#     # 6. FFmpeg 명령어를 실행하여 비디오 생성
#     command = [
#         'ffmpeg',
#         '-framerate', '1/2',
#         '-i', os.path.join(temp_folder, 'img%03d.jpeg'),
#         # ⭐️ 수정된 부분: 홀수 크기 문제를 해결하기 위해 scale 필터 추가
#         '-vf', 'scale=trunc(iw/2)*2:trunc(ih/2)*2',
#         '-r', '30',
#         '-pix_fmt', 'yuv420p',
#         '-y',
#         output_video
#     ]

#     try:
#         subprocess.run(command, check=True, capture_output=True, text=True)
#         print("\n🎉 영상 생성 완료! 🎉")
#         print("="*30)
#         print(f"🎬 AI 추천 제목: {video_title}")
#         print(f"💾 저장된 파일: {output_video}")
#         print("="*30)
#         print("왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.")
#     except subprocess.CalledProcessError as e:
#         print("\n❗️ FFmpeg 실행 중 오류가 발생했습니다.")
#         print(e.stderr)

#     # 7. 임시 작업 폴더 삭제
#     shutil.rmtree(temp_folder)


🎉 영상 생성 완료! 🎉
🎬 AI 추천 제목: 여름 친구들과의 추억
💾 저장된 파일: output_shorts_with_title.mp4
왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.


In [ ]:
# import os
# import glob
# import subprocess
# import shutil
# import google.generativeai as genai
# # ⭐️ Colab 보안 비밀(Secrets)을 사용하기 위한 라이브러리
# from google.colab import userdata

# # --- 설정 ---
# # 1. ⭐️ Colab 보안 비밀에서 API 키를 안전하게 가져옵니다.
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# # 2. 영상의 주제를 간단하게 입력하세요.
# VIDEO_THEME = "친구들과 함께한 즐거운 여름 휴가"

# # 3. 사진이 저장된 구글 드라이브 폴더 경로
# image_folder = '/content/drive/MyDrive/images'

# # 4. 영상을 만들기 위한 임시 작업 폴더
# temp_folder = '/content/temp_images'

# # 5. 최종적으로 저장될 영상 파일 이름
# output_video = 'output_shorts_with_title.mp4'


# # --- 로직 ---
# print("🚀 영상 생성을 시작합니다...")

# # 1. Gemini API 설정
# model = None
# if GEMINI_API_KEY:
#     try:
#         genai.configure(api_key=GEMINI_API_KEY)
#         model = genai.GenerativeModel('gemini-1.5-flash')
#         print("✅ Gemini API 설정 완료.")
#     except Exception as e:
#         print(f"❗️ API 설정 중 오류 발생: {e}")
# else:
#     print("❗️ Colab 보안 비밀(Secrets)에서 'GEMINI_API_KEY'를 찾을 수 없습니다.")


# # 2. AI로 영상 제목 생성
# video_title = "제목 생성 실패"
# if model:
#     try:
#         print("🤖 AI가 영상 제목을 생성 중입니다...")
#         prompt = f"'{VIDEO_THEME}'라는 주제의 쇼츠 영상에 어울리는, 짧고 임팩트 있는 제목 하나만 추천해줘."
#         response = model.generate_content(prompt)
#         video_title = response.text.strip().replace("*", "").replace("\"", "")
#     except Exception as e:
#         print(f"❗️ AI 제목 생성 중 오류 발생: {e}")

# # 3. 이전 작업 파일이 남아있다면 깨끗하게 정리
# if os.path.exists(temp_folder):
#     shutil.rmtree(temp_folder)
# os.makedirs(temp_folder)

# # 4. 지정된 폴더에서 모든 .jpeg 파일 목록을 가져와 정렬
# image_files = sorted(glob.glob(os.path.join(image_folder, '*.jpeg')))

# if not image_files:
#     print(f"❗️'{image_folder}' 폴더에 .jpeg 파일이 없습니다. 경로를 확인해주세요.")
# else:
#     # 5. FFmpeg가 인식하기 쉽도록 파일을 숫자 순서대로 복사
#     for i, file_path in enumerate(image_files):
#         shutil.copy(file_path, os.path.join(temp_folder, f"img{i:03d}.jpeg"))

#     # 6. FFmpeg 명령어를 실행하여 비디오 생성
#     command = [
#         'ffmpeg', '-framerate', '1/2', '-i', os.path.join(temp_folder, 'img%03d.jpeg'),
#         '-vf', 'scale=trunc(iw/2)*2:trunc(ih/2)*2', '-r', '30',
#         '-pix_fmt', 'yuv420p', '-y', output_video
#     ]

#     try:
#         subprocess.run(command, check=True, capture_output=True, text=True)
#         print("\n🎉 영상 생성 완료! 🎉")
#         print("="*30)
#         print(f"🎬 AI 추천 제목: {video_title}")
#         print(f"💾 저장된 파일: {output_video}")
#         print("="*30)
#         print("왼쪽 파일 탐색기에서 새로고침하여 확인 후 다운로드할 수 있습니다.")
#     except subprocess.CalledProcessError as e:
#         print("\n❗️ FFmpeg 실행 중 오류가 발생했습니다.")
#         print(e.stderr)

#     # 7. 임시 작업 폴더 삭제
#     shutil.rmtree(temp_folder)

SecretNotFoundError: Secret GEMINI_API_KEY does not exist.